In [1]:
from bs4 import BeautifulSoup
from io import BytesIO

import csv
import datetime as dt
import numpy as np
import pandas as pd
import requests
import time

#參考: 助教的Github; https://www.finlab.tw/Python-時間序列實作！

# #STEP 1: 整理要爬的ETF（手動）
#  = ['VGK' , 'EZU' , 'HEDJ' , 'IEUR' , 'FEZ' , 'EWG' , 'IEV' , 'EWU' , 'HEZU' , 'EWP' , 'EWL' , 'DBEU' , 'DFE' , 'EUFN' , 'FEP' , 'EWQ' , 
#               'FDD' , 'GREK' ,'EWI' , 'EWD' , 'HEWG' , 'FEU' , 'IEUS' , 'FSZ' , 'GVAL' , 'FGM' , 'EWN' , 'EUSC' , 'NORW' , 'EWO' , 'EDEN' , 'EIRL' , 'FEUZ' , 
#               'DXGE' , 'EWUS' , 'EWK' , 'EURL' , 'ADRD' , 'EUDG' , 'EWGS' , 'WBIA' , 'DBEZ' , 'ENOR' , 'EUMV' , 'EFNL' , 'OEUR' , 'PGAL' , 'GXF' , 
#               'DBGR' , 'JPEU' , 'SMEZ' , 'EPV' , 'HEWU' , 'DAX' , 'ADRU' , 'HEWP' , 'EUDV' , 'HFXE' , 'ZGBR' , 'FKU' , 'ZDEU' , 'UPV' , 'HEWL' , 'SCID' , 
#               'EDOM' , 'DBUK' , 'GRMY' , 'HEWI' , 'EURZ']

#STEP 1: 整理要爬的ETF
file = pd.read_csv('ETF_data.csv')
file['Date'] = pd.to_datetime(file['Inception'])
file = file.loc[file['Date'] < dt.datetime(2015,12,1)]
crawl_list = file['Symbol'].tolist()



#STEP 2: 做一個爬yahoo finance上指定ETF資料的function
today = int(time.time())
today = format(today)
def crawl_price(ETF): 
    #下載自201512/31 0:00(GMT)到程式執行當下的資料
    dl = requests.post('https://query1.finance.yahoo.com/v7/finance/download/' + ETF + '?period1=1448985600&period2=' + today + '&interval=1mo&events=history&crumb=PapEKXpAl4Y')
    df = pd.read_csv(BytesIO(dl.content))
    return df

#印出前五行確定有抓到資料
test = crawl_price('VGK')
test.head()

Date       Open       High        Low      Close  Adj Close  \
0  2015-12-01  51.730000  51.840000  48.770000  49.880001  44.986820   
1  2016-01-01  49.139999  49.180000  43.799999  47.110001  42.663246   
2  2016-02-01  46.680000  47.220001  43.470001  45.590000  41.286716   
3  2016-03-01  46.330002  49.209999  46.220001  48.520000  43.940159   
4  2016-04-01  47.680000  50.549999  47.040001  49.860001  45.408333   

      Volume  
0  121472400  
1  135790300  
2  125209500  
3   98976200  
4   84964300

In [2]:
#STEP 3: 建立只有Adj Close的dataframe
def crawl_close(ETF): 
    #下載自201512/31 0:00(GMT)到程式執行當下的資料
    dl = requests.post('https://query1.finance.yahoo.com/v7/finance/download/' + ETF + '?period1=1448985600&period2=' + today + '&interval=1mo&events=history&crumb=PapEKXpAl4Y')
    df = pd.read_csv(BytesIO(dl.content))
    df.set_index('Date' , inplace = True)
    
    #為了讀取Adj Close的欄位把空格去除掉
    df.columns = [c.replace(' ', '') for c in df.columns]
    df_close = df.AdjClose
    df_close = df_close.to_frame()
    
    #把欄位名稱改成ETF的名稱
    df_close = df_close.rename(columns = {'AdjClose' : ETF})
    #df_close.reset_index(inplace = True)
    
    return df_close

#印出前五行確定有抓到資料
test = crawl_close('VGK')
test.head()

VGK
Date                 
2015-12-01  44.986820
2016-01-01  42.663246
2016-02-01  41.286716
2016-03-01  43.940159
2016-04-01  45.408333

In [3]:
#STEP 4: 全部放在一起
final_df = crawl_close('VGK')
print('正在下載資料: VGK')
for ETFs in crawl_list :
    if ETFs != 'VGK' :
        print('正在下載資料: ' + ETFs)
        next_df = crawl_close(ETFs)
        final_df = pd.merge(final_df , next_df , how = 'left' , on = 'Date')

正在下載資料: VGK
正在下載資料: EZU
正在下載資料: HEDJ
正在下載資料: IEUR
正在下載資料: FEZ
正在下載資料: EWG
正在下載資料: IEV
正在下載資料: EWU
正在下載資料: HEZU
正在下載資料: EWP
正在下載資料: EWL
正在下載資料: DBEU
正在下載資料: DFE
正在下載資料: EUFN
正在下載資料: FEP
正在下載資料: EWQ
正在下載資料: FDD
正在下載資料: GREK
正在下載資料: EWI
正在下載資料: EWD
正在下載資料: HEWG
正在下載資料: FEU
正在下載資料: IEUS
正在下載資料: FSZ
正在下載資料: GVAL
正在下載資料: FGM
正在下載資料: EWN
正在下載資料: EUSC
正在下載資料: NORW
正在下載資料: EWO
正在下載資料: EDEN
正在下載資料: EIRL
正在下載資料: FEUZ
正在下載資料: DXGE
正在下載資料: EWUS
正在下載資料: EWK
正在下載資料: EURL
正在下載資料: ADRD
正在下載資料: EUDG
正在下載資料: EWGS
正在下載資料: WBIA
正在下載資料: DBEZ
正在下載資料: ENOR
正在下載資料: EUMV
正在下載資料: EFNL
正在下載資料: OEUR
正在下載資料: PGAL
正在下載資料: GXF
正在下載資料: DBGR
正在下載資料: SMEZ
正在下載資料: EPV
正在下載資料: HEWU
正在下載資料: DAX
正在下載資料: ADRU
正在下載資料: HEWP
正在下載資料: EUDV
正在下載資料: HFXE
正在下載資料: ZGBR
正在下載資料: FKU
正在下載資料: ZDEU
正在下載資料: UPV
正在下載資料: HEWL
正在下載資料: EDOM
正在下載資料: DBUK
正在下載資料: GRMY
正在下載資料: HEWI
正在下載資料: EURZ


In [4]:
#STEP 5: 加回index並印出結果
# final_df.reset_index(inplace = True)
final_df

VGK        EZU       HEDJ       IEUR        FEZ        EWG  \
Date                                                                           
2015-12-01  44.986820  32.259003  46.556320  38.280113  31.261314  24.428692   
2016-01-01  42.663246  30.632404  48.156239  36.239799  29.478609  22.693775   
2016-02-01  41.286716  29.340677  46.294460  35.135811  28.211199  21.789011   
2016-03-01  43.940159  31.730375  48.091396  37.480633  30.299227  23.999626   
2016-04-01  45.408333  32.542316  48.613300  38.684975  31.151741  24.466000   
2016-05-01  45.180660  31.361313  49.755505  38.511627  30.959896  24.186176   
2016-06-01  42.494045  29.774330  46.923210  36.029945  28.475069  22.432608   
2016-07-01  44.844322  31.990591  50.217289  38.102268  30.536980  24.508524   
2016-08-01  45.150837  32.349918  50.966515  38.270287  30.799498  25.043604   
2016-09-01  45.262299  32.614704  50.966515  38.615650  30.715117  25.100933   
2016-10-01  43.900677  32.283733  51.668884  37.346199  30.663122  24.756954   
2016-11-01  42.873432  30.997675  51.164753  36.552792  29.542795  23.725016   
2016-12-01  44.769165  32.718712  54.598568  37.980927  31.501011  25.301588   
2017-01-01  46.315792  33.636856  54.834244  39.461731  32.129837  26.294836   
2017-02-01  46.587917  33.703476  56.906166  39.706543  32.290390  26.304392   
2017-03-01  48.389500  35.787941  59.971077  41.401394  34.330372  27.480061   
2017-04-01  50.548161  37.348904  61.999271  43.025616  35.798573  28.368982   
2017-05-01  53.029335  39.195408  62.439129  45.120644  37.416752  29.649792   
2017-06-01  52.019882  38.386368  59.599247  44.131977  36.404209  28.932922   
2017-07-01  54.249599  40.242233  60.355698  46.136265  38.231300  29.919529   
2017-08-01  54.287872  40.435566  60.083168  46.107548  38.279427  29.968245   
2017-09-01  55.809162  41.875896  62.973930  47.562767  39.703957  31.595259   
2017-10-01  56.290031  42.368896  64.658646  47.668079  40.226559  32.218784   
2017-11-01  56.261215  42.262562  62.896381  47.160664  39.975441  32.384411   
2017-12-01  56.818352  41.933899  62.029850  48.031883  39.318680  32.170071   
2018-01-01  60.338043  45.069000  64.386024  51.291782  42.181545  34.194489   
2018-02-01  56.629692  42.369907  61.652008  48.234207  39.284836  31.674587   
2018-03-01  56.156490  42.088348  61.241905  48.069721  39.013573  31.293671   
2018-04-01  57.643791  43.039829  63.541138  49.056656  40.378094  31.703886   
2018-05-01  56.246368  41.496101  63.805122  47.914906  38.446545  30.902988   
2018-06-01  54.470470  39.816452  61.732384  46.037788  37.204140  29.281654   
2018-07-01  57.440029  42.339890  65.589996  48.842056  39.703148  31.309999   
2018-08-01  55.816978  40.997189  63.820000  47.671356  37.995911  30.230000   
2018-09-01  55.638840  40.788322  63.220001  47.582066  37.985989  29.740000   
2018-10-01  51.443596  37.167999  59.700001  43.831852  34.921246  27.250000   
2018-11-01  51.095871  37.038704  59.860001  43.563980  34.861622  26.840000   
2018-12-01  48.304127  34.870487  56.439999  41.004314  33.062889  25.350000   
2019-01-01  51.869999  37.349998  60.290001  44.070000  35.161732  26.790001   
2019-02-01  53.509998  38.520000  62.689999  45.509998  36.338776  27.299999   
2019-03-01  54.660000  39.270000  64.639999  46.450001  37.086899  27.580000   
2019-03-15  54.660000  39.270000  64.639999  46.450001  37.180000  27.580000   

                  IEV        EWU       HEZU        EWP    ...           ZGBR  \
Date                                                      ...                  
2015-12-01  36.600449  28.592768  23.662924  24.951370    ...            NaN   
2016-01-01  34.837933  27.481667  22.577198  23.688608    ...            NaN   
2016-02-01  33.754978  26.622868  21.576420  22.502380    ...            NaN   
2016-03-01  35.893349  28.054205  22.356844  24.425507    ...            NaN   
2016-04-01  36.994656  29.181381  22.788372  25.917274    ...            NaN   
2016-05-01  36.68